In [17]:
from sqlalchemy import create_engine
# https://sqlalchemy-utils.readthedocs.io/en/latest/index.html
from sqlalchemy_utils.functions import database_exists, create_database

# różne connection strings dla różnych baz
# więcej przykładów na https://docs.sqlalchemy.org/en/14/core/engines.html#database-urls

# sqlite
# baza w piku, linux/mac
# SQLITE_DATABASE_URL="sqlite://///home/jakub/baza2.db"
SQLITE_DATABASE_URL=r"sqlite:///C:\PythonScripts\SpyderScripts\Jakub Tomczak\lab2\Base.db"

def create_database_if_doesn_exist(engine):
    if not database_exists(engine.url):
        create_database(engine.url)

def create_connection(connection_url: str, debug_sql: bool = True):
    url = connection_url
    engine = create_engine(url, echo=debug_sql)

    # creates database if doesn't exist
    create_database_if_doesn_exist(engine)
    return engine

In [18]:
engine = create_connection(connection_url=SQLITE_DATABASE_URL, debug_sql=False)


In [19]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

In [23]:
from sqlalchemy import Column, Integer, String
from typing import List
from pydantic import BaseModel

class User(Base):
    # specjalny atrybut (ważna jest jej nazwa - taka konwencja),
    # informuje SQLAlchemy na jaką tablicę ma mapować klasę User
    __tablename__ = "users"

    # id użytkownika, jest kluczem podstawowym, co oznacza, że
    # żaden inny użytkownik nie może mieć takiego samego id
    id = Column(Integer, primary_key=True, index=True, unique=True)
    first_name = Column(String(30))
    last_name = Column(String(30))
    age = Column(Integer)

    # reprezentacja naszej klasy = metoda podobna do __str__
    def __repr__(self):
        return f'<User(id={self.id}, first_name={self.first_name}, last_name={self.last_name}, age={self.age})>'

In [24]:
User.__table__


Table('users', MetaData(bind=None), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('first_name', String(length=30), table=<users>), Column('last_name', String(length=30), table=<users>), Column('age', Integer(), table=<users>), schema=None)

In [25]:
użytkownik = {
    "first_name": "Jan",
    "last_name": "Kowalski",
    "age": 21
}
jan = User(**użytkownik)
print(jan)

jan = User(first_name="Jan", last_name="Kowalski", age=20)
print(jan)


<User(id=None, first_name=Jan, last_name=Kowalski, age=21)>
<User(id=None, first_name=Jan, last_name=Kowalski, age=20)>


In [26]:
# tworzy tabele z listy tabels jeżeli nie istniały w bazie
tables = [User.__table__]
Base.metadata.create_all(engine, tables)

In [27]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session(autocommit=False)


In [28]:
print(jan)
session.add(jan)
# użytkownik nie jest jeszcze dodany do bazy
# session.new przechowuje zbiór obiektów które mają być doane do bazy
session.new

<User(id=None, first_name=Jan, last_name=Kowalski, age=20)>


IdentitySet([<User(id=None, first_name=Jan, last_name=Kowalski, age=20)>])

In [29]:
session.commit()
print(jan)

<User(id=1, first_name=Jan, last_name=Kowalski, age=20)>


In [30]:
users = [
    ('Andrzej', 'Nowak', 10),
    ('Janina', 'Nowak', 12),
    ('Anna', 'Kowalska', 33),
    ('Jan', 'Nowak', 30)
]
session.add_all([User(first_name=f_name, last_name=l_name, age=age) for f_name, l_name, age in users])
session.commit()

In [31]:
users = session.query(User).all()
for user in users:
    print(f'ID: {user.id}, Imię {user.first_name}, nazwisko {user.last_name}')


ID: 1, Imię Jan, nazwisko Kowalski
ID: 2, Imię Andrzej, nazwisko Nowak
ID: 3, Imię Janina, nazwisko Nowak
ID: 4, Imię Anna, nazwisko Kowalska
ID: 5, Imię Jan, nazwisko Nowak


In [32]:
# pobieranie wszystkich wyników
users = session.query(User).all()
print(users)

[<User(id=1, first_name=Jan, last_name=Kowalski, age=20)>, <User(id=2, first_name=Andrzej, last_name=Nowak, age=10)>, <User(id=3, first_name=Janina, last_name=Nowak, age=12)>, <User(id=4, first_name=Anna, last_name=Kowalska, age=33)>, <User(id=5, first_name=Jan, last_name=Nowak, age=30)>]


In [33]:
# sortowanie danych
users_sorted_by_last_first_name = session.query(User).order_by(User.first_name)
for user in users_sorted_by_last_first_name:
    print(f'{user.id}:\t{user.first_name}')

2:	Andrzej
4:	Anna
1:	Jan
5:	Jan
3:	Janina


In [34]:
# filtrowanie wyników - pobranie tylko nazwisk użytkowników
last_names = session.query(User.last_name).all()
print('wszystkie nazwiska\n', last_names)
print('*'*20)
# lepiej wykonywać jak najwięcej możliwych operacji po stronie bazy danych
# w ramach zapytania - mniej danych do przesyłania
last_names = session.query(User.last_name).distinct().all()
print('wszystkie unikatowe nazwiska\n', last_names)

wszystkie nazwiska
 [('Kowalski',), ('Nowak',), ('Nowak',), ('Kowalska',), ('Nowak',)]
********************
wszystkie unikatowe nazwiska
 [('Kowalski',), ('Nowak',), ('Kowalska',)]


In [35]:
# typ zwracanych rezultatów to result
last_names = session.query(User.last_name).distinct().all()
print('iterowanie po rezultatach')
for last_name in last_names:
    print(type(last_name), '\t', last_name)

print('*'*20)
for last_name, in last_names:
    print(type(last_name), '\t', last_name)

iterowanie po rezultatach
<class 'sqlalchemy.util._collections.result'> 	 ('Kowalski',)
<class 'sqlalchemy.util._collections.result'> 	 ('Nowak',)
<class 'sqlalchemy.util._collections.result'> 	 ('Kowalska',)
********************
<class 'str'> 	 Kowalski
<class 'str'> 	 Nowak
<class 'str'> 	 Kowalska


In [36]:
women_in_last_names = session.query(User).filter(User.first_name.like('%a'))
print('Kobiety w bazie danych użytkowników')
for user in women_in_last_names:
    print(f'{user.id}:\t{user.first_name}')

# możliwość użycia operatora _in
print('Andrzeje i Janiny w bazie danych')
filtered_query = session.query(User).filter(User.first_name.in_(['Andrzej', 'Janina']))
for user in filtered_query:
    print(f'{user.id}:\t{user.first_name}')

print('Andrzeje i Janiny w bazie danych')
and_conditions = session.query(User).filter(User.first_name == 'Jan', User.last_name == 'Kowalski')
for user in and_conditions:
    print(f'{user.id}:\t{user.first_name} {user.last_name}')

Kobiety w bazie danych użytkowników
3:	Janina
4:	Anna
Andrzeje i Janiny w bazie danych
2:	Andrzej
3:	Janina
Andrzeje i Janiny w bazie danych
1:	Jan Kowalski


In [37]:
session.rollback()
print('Aktualizacja użytkowników')
# chcemy znaleźć pierwszego jana w tabeli i go zaktualizować
user_to_be_updated = session.query(User).filter(User.first_name=='Jan').first()
print('przed aktualizacją', user_to_be_updated)
user_to_be_updated.first_name = 'Janeczek'
session.add(user_to_be_updated)
session.commit()

# po aktualizacji nie możemy go znaleźć po imieniu, bo je zmieniliśmy na janeczek
# więc używamy jego id
id = user_to_be_updated.id
user_to_be_updated = session.query(User).filter(User.id == id).first()
print('po aktualizacji', user_to_be_updated)

Aktualizacja użytkowników
przed aktualizacją <User(id=1, first_name=Jan, last_name=Kowalski, age=20)>
po aktualizacji <User(id=1, first_name=Janeczek, last_name=Kowalski, age=20)>


In [38]:
# usuwanie użytkowników
user_to_be_deleted = session.query(User).filter(User.first_name == 'Jan').first()
print('przed usunięciem', user_to_be_deleted)

if type(user_to_be_deleted) is list:
    for r in user_to_be_deleted:
        session.delete(r)
        session.commit()
else:
    session.delete(user_to_be_deleted)
    session.commit()

user_to_be_deleted = session.query(User).filter(User.first_name == 'Jan').first()
print('po usunięciu', user_to_be_deleted)

przed usunięciem <User(id=5, first_name=Jan, last_name=Nowak, age=30)>
po usunięciu None
